In [69]:
%env WORKDIR=/tmp/vault
%env PROJECT=hc-73f8ab4df44146ba8cffb466d83
%env KEYRING_NAME=kmse-test
%env KEYRING_GKMS=gcks-test
%env LOCATION=global
%env KEY_NAME=testkey1

env: WORKDIR=/tmp/vault
env: PROJECT=hc-73f8ab4df44146ba8cffb466d83
env: KEYRING_NAME=kmse-test
env: KEYRING_GKMS=gcks-test
env: LOCATION=global
env: KEY_NAME=testkey1


In [5]:
%%bash
cat $WORKDIR/VAULT_TOKEN.TXT
cat $WORKDIR/VAULT_ADDR.TXT

hvs.QbtInWOOYqrZJnMrDmcaArKs
https://vault-primary-europe-southwest1-jlhb.josemerchan-f986bf.gcp.sbx.hashicorpdemo.com:8200


In [6]:
%env VAULT_ADDR=https://vault-primary-europe-southwest1-jlhb.josemerchan-f986bf.gcp.sbx.hashicorpdemo.com:8200
%env VAULT_TOKEN=hvs.QbtInWOOYqrZJnMrDmcaArKs

env: VAULT_ADDR=https://vault-primary-europe-southwest1-jlhb.josemerchan-f986bf.gcp.sbx.hashicorpdemo.com:8200
env: VAULT_TOKEN=hvs.QbtInWOOYqrZJnMrDmcaArKs


# [KMSE](https://developer.hashicorp.com/vault/docs/secrets/key-management) and [KMSE API](https://developer.hashicorp.com/vault/api-docs/secret/key-management#key-management-secrets-engine-api)

In [45]:
! vault secrets enable keymgmt

Success! Enabled the keymgmt secrets engine at: keymgmt/


In [ ]:
! vault write keymgmt/key/aes256-gcm96 type="aes256-gcm96"
! vault write keymgmt/key/ecdsa-p256 type="ecdsa-p256"

Success! Data written to: keymgmt/key/aes256-gcm96
Success! Data written to: keymgmt/key/ecdsa-p256


In [65]:
! vault write keymgmt/key/aes256-key2 type="aes256-gcm96"

Success! Data written to: keymgmt/key/aes256-key2


In [47]:
! vault read -format=json keymgmt/key/aes256-gcm96 | jq

{
  "request_id": "8d3a5e53-034d-f21d-e295-5f4bfc9f308d",
  "lease_id": "",
  "lease_duration": 0,
  "renewable": false,
  "data": {
    "deletion_allowed": false,
    "latest_version": 1,
    "min_enabled_version": 1,
    "name": "aes256-gcm96",
    "type": "aes256-gcm96",
    "versions": {
      "1": {
        "creation_time": "2024-11-18T14:49:12.077439421Z"
      }
    }
  },
  "warnings": null
}


## Credentials must be copied to the Vault node and run the command

In [ ]:
! kubectl cp $WORKDIR/creds.json -n vault vault-primary-0:/tmp/creds.json
# ! kubectl cp $WORKDIR/creds.json -n vault vault-primary-1:/tmp/creds.json
# ! kubectl cp $WORKDIR/creds.json -n vault vault-primary-2:/tmp/creds.json
# ! kubectl cp $WORKDIR/creds.json -n vault vault-primary-3:/tmp/creds.json
# ! kubectl cp $WORKDIR/creds.json -n vault vault-primary-4:/tmp/creds.json

Defaulted container "vault" out of: vault, auditlog-rotator


[Key compatibility with GCP](https://developer.hashicorp.com/vault/docs/secrets/key-management/gcpkms#key-purpose-compatability) and [GCP Setup Guide](https://developer.hashicorp.com/vault/docs/secrets/gcpkms)

In [71]:
%%bash

vault write keymgmt/kms/gcpckms \
    provider="gcpckms" \
    key_collection="projects/${PROJECT}/locations/${LOCATION}/keyRings/${KEYRING_NAME}" \
    credentials=service_account_file=/tmp/creds.json

Success! Data written to: keymgmt/kms/gcpckms


In [63]:
%%bash

vault write keymgmt/kms/gcpckms/key/aes256-gcm96 \
    purpose="encrypt,decrypt" \
    protection="software"


Success! Data written to: keymgmt/kms/gcpckms/key/aes256-gcm96


In [64]:
%%bash

vault write keymgmt/kms/gcpckms/key/ecdsa-p256 \
    purpose="sign" \
    protection="software"


Success! Data written to: keymgmt/kms/gcpckms/key/ecdsa-p256


In [ ]:
%%bash

vault write keymgmt/kms/gcpckms/key/aes256-key2 \
    purpose="encrypt,decrypt" \
    protection="software"

In [66]:
! vault list keymgmt/kms/gcpckms/key/

Keys
----
aes256-gcm96
ecdsa-p256


In [67]:
! vault write -f keymgmt/key/aes256-gcm96/rotate

Success! Data written to: keymgmt/key/aes256-gcm96/rotate


In [68]:
!  vault read -format=json keymgmt/key/aes256-gcm96 | jq 

{
  "request_id": "0a5b02cc-cfff-e641-4bc7-4d19e17dda33",
  "lease_id": "",
  "lease_duration": 0,
  "renewable": false,
  "data": {
    "deletion_allowed": false,
    "latest_version": 2,
    "min_enabled_version": 1,
    "name": "aes256-gcm96",
    "type": "aes256-gcm96",
    "versions": {
      "1": {
        "creation_time": "2024-11-18T14:49:12.077439421Z"
      },
      "2": {
        "creation_time": "2024-11-18T15:12:13.987094497Z"
      }
    }
  },
  "warnings": null
}


> This [engine](https://developer.hashicorp.com/vault/docs/secrets/gcpkms#faq) was created before the functionality of [managed keys](https://developer.hashicorp.com/vault/docs/enterprise/managed-keys)